In [1]:
import pandas as pd
import simpy
import numpy as np
import random

In [2]:
class Balancer(object):
    def __init__(self, env, count, flag_RR):
        self.env = env
        self.count = count
        self.servers = [simpy.Resource(env) for x in range(count)]
        self.flag_RR = flag_RR
        self.nextServer = 0

    def serve(self, client):
        if self.flag_RR:
            server = self.select_server_RR()
        else:
            server = self.select_server_not_bussy() 
        global queue
        if (queue < len(server.queue)):
            queue = len(server.queue)
        with server.request() as req:
            yield req
            yield self.env.timeout(client.get_service_time())

    def select_server_RR(self):
        current = self.nextServer
        self.nextServer += 1
        if (current == 4):
            self.nextServer = 0

        return self.servers[current]
    
    def select_server_not_bussy(self):
        min_server = self.servers[0]
        for server in self.servers:
            if (len(server.queue) < len(min_server.queue)):
                min_server = server 

        return min_server


class Client(object):

    def __init__(self, env):
        self.env = env
        self.type = np.random.choice(['A', 'B', 'C'], p=[0.7, 0.2, 0.1])
        self.service_time =  self.service_time = {
            'A': random.uniform(60,180),  
            'B': random.uniform(120,360),
            'C': random.uniform(200,800)
        }.get(self.type)
        self.arrive = 0
        
    def release(self, balancer):
        self.arrive = self.env.now
        yield self.env.process(balancer.serve(self))
        wait = self.env.now - self.arrive
        global max_wait_time
        if (max_wait_time < wait):
            max_wait_time = wait
        print(f"%.2f Type {self.type}  attended" % (self.env.now))
        
    def get_service_time(self):
        return self.service_time


servers = 5
arrival_rate = 0.045
client_count = 100
queue = 0
max_wait_time = 0

def generate_clients(environment, count, interval, checkout):
    for i in range(count):
        client = Client(env)
        environment.process(client.release(checkout))
        yield environment.timeout(np.random.exponential(interval))


### Round Robin version

In [3]:
env = simpy.Environment()
balancer = Balancer(env, servers, True)
env.process(generate_clients(env, client_count, arrival_rate, balancer))
env.run()

print(f"Max queue: {queue}" )
print(f"wait time maximun: {max_wait_time}")

66.30 Type A  attended
70.09 Type A  attended
88.94 Type A  attended
101.01 Type A  attended
142.96 Type A  attended
208.18 Type A  attended
249.09 Type A  attended
302.86 Type B  attended
345.45 Type C  attended
368.58 Type B  attended
377.50 Type A  attended
421.54 Type A  attended
424.14 Type A  attended
463.13 Type A  attended
486.63 Type A  attended
487.04 Type A  attended
528.61 Type A  attended
542.78 Type A  attended
582.49 Type A  attended
602.36 Type A  attended
626.96 Type A  attended
661.15 Type A  attended
675.27 Type A  attended
714.23 Type A  attended
771.90 Type A  attended
835.79 Type A  attended
853.42 Type A  attended
860.73 Type A  attended
951.34 Type B  attended
990.99 Type A  attended
1008.28 Type B  attended
1117.10 Type B  attended
1146.38 Type A  attended
1273.08 Type B  attended
1317.62 Type B  attended
1329.27 Type C  attended
1416.76 Type C  attended
1416.91 Type A  attended
1457.31 Type A  attended
1488.49 Type A  attended
1604.81 Type B  attended
1609.80 

### Least busy server

In [4]:
env = simpy.Environment()
balancer = Balancer(env, servers, False)
env.process(generate_clients(env, client_count, arrival_rate, balancer))
env.run()

print(f"Max queue: {queue}" )
print(f"wait time maximun: {max_wait_time}")

72.65 Type A  attended
86.17 Type A  attended
137.99 Type A  attended
173.08 Type A  attended
225.86 Type A  attended
310.96 Type B  attended
407.46 Type C  attended
519.43 Type B  attended
571.72 Type B  attended
597.62 Type C  attended
640.49 Type A  attended
640.86 Type B  attended
684.90 Type A  attended
704.43 Type C  attended
744.67 Type A  attended
747.69 Type A  attended
820.92 Type B  attended
841.67 Type A  attended
1058.69 Type C  attended
1072.32 Type B  attended
1107.02 Type B  attended
1144.98 Type B  attended
1216.41 Type A  attended
1261.85 Type A  attended
1359.97 Type A  attended
1368.80 Type C  attended
1384.37 Type C  attended
1413.53 Type B  attended
1459.63 Type B  attended
1530.68 Type A  attended
1532.91 Type A  attended
1562.94 Type A  attended
1634.01 Type A  attended
1639.16 Type A  attended
1679.25 Type A  attended
1693.92 Type A  attended
1694.07 Type B  attended
1762.39 Type A  attended
1782.31 Type A  attended
1813.92 Type A  attended
1886.39 Type A  atte